In [73]:
pip install selenium webdriver-manager fake_useragent 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager


Adjusting the URL stem

In [2]:
from urllib.parse import urlparse, urljoin
yelp_url = "https://www.yelp.com/biz/mix-kitchen-and-bar-ithaca-11?osq=Reservations"
def extract_business_slug(yelp_url):
    """Extracts the business slug from a Yelp URL"""
    parsed_url = urlparse(yelp_url)
    path_parts = parsed_url.path.split("/")
    
    if len(path_parts) > 2 and path_parts[1] == "biz":
        business_slug = path_parts[2]  
        clean_url = urljoin("https://www.yelp.com", f"/biz/{business_slug}")  
    return clean_url

clean_yelp_url = extract_business_slug(yelp_url)
clean_yelp_url

'https://www.yelp.com/biz/mix-kitchen-and-bar-ithaca-11'

In [3]:
from fake_useragent import UserAgent

ua = UserAgent()
headers = {"User-Agent": ua.random}
page = requests.get(clean_yelp_url, headers=headers)

In [4]:
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [8]:
import time
import random 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager

headers = {"User-Agent": ua.random}
def get_driver():
    """This function is used to set up a Selenium WebDriver with headless Chrome in order to go through the data within Yelp."""
    options = Options() 
    options.add_argument("--headless")   
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")     
    options.add_argument("--window-size=1920,1080")
    time.sleep(5)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def click_next_page(driver):
    """Clicks the 'Next' button to load the next page of reviews, if available."""
    try:
        time.sleep(2)  # <--- Short delay to allow page update

        next_button = driver.find_element(By.XPATH, "//a[contains(@class, 'next-link')]")

        if next_button:
            driver.execute_script("arguments[0].scrollIntoView();", next_button)  # Scroll to button
            time.sleep(2)
            driver.execute_script("arguments[0].click();", next_button)  
            print("Clicked 'Next' button, loading next page...")
            time.sleep(4)  
            return True  

    except (StaleElementReferenceException, NoSuchElementException):
        print("No more pages or 'Next' button not found.")
        return False  

    return False

def scrape_yelp_reviews(clean_yelp_url):
    "This function extracts each yelp rating using Selenium"
    time.sleep(random.uniform(7, 14)) # <-- allows time for javascript to load
    driver = get_driver()
    driver.get(clean_yelp_url)
    all_reviews = []
    page = 1
    while True:
        time.sleep(random.uniform(2, 5))
        review_sections = driver.find_elements(By.XPATH, "//ul[contains(@class, 'list__09f24__ynIEd')]//li") # <--- tells the webscraper to start scraping at the start constumer reviews section
        print("review" + str(review_sections))
        time.sleep(random.uniform(3, 7))
        print(f"Found {len(review_sections)} reviews.")  # Debugging
        if not review_sections:
            print("No reviews found on this page.")
            break 
        
        page_reviews = []
        for review in review_sections:
            try:
                #extract aspects of review
                reviewer_name = review.find_element(By.XPATH, ".//a[contains(@class, 'css-1lx1e1r')]").text
                reviewer_location = review.find_element(By.XPATH, ".//span[contains(@class, 'css-qgunke')]").text
                rating_element = review.find_elements(By.XPATH, ".//div[contains(@class, 'y-css-dnttlc')]")  
                if rating_element:
                    rating = rating_element.get_attribute("aria-label")  
                else: 
                    continue
                review_text = review.find_element(By.XPATH, ".//p[contains(@class, 'raw__09f24__T4Ezm')]").text
                review_date = review.find_element(By.XPATH, ".//span[contains(@class, 'y-css-1d8mpv1')]").text
                
                #counts 
                helpful = review.find_element(By.XPATH, ".//span[contains(@class, 'y-css-ghxju8') and text()='Helpfuk']/following-sibling::span[contains(@class, 'y-css-7nl72w')]").text
                thanks = review.find_element(By.XPATH, ".//span[contains(@class, 'y-css-ghxju8') and text()='Thanks']/following-sibling::span[contains(@class, 'y-css-7nl72w')]").text
                love = review.find_element(By.XPATH, ".//span[contains(@class, 'y-css-ghxju8') and text()='Love this']/following-sibling::span[contains(@class, 'y-css-7nl72w')]").text
                oh_no = review.find_element(By.XPATH, ".//span[contains(@class, 'y-css-ghxju8') and text()='Oh no']/following-sibling::span[contains(@class, 'y-css-7nl72w')]").text
                
                # Store all extracted data
                review_data = {
                    "Reviewer Name": reviewer_name,
                    "Location": reviewer_location,
                    "Star Rating": rating,
                    "Review Text": review_text,
                    "Date": review_date,
                    "Helpful": helpful,
                    "Thanks": thanks,
                    "Love": love,
                    "Oh No": oh_no
                }
                page_reviews.append(review_data)
            except Exception as e:
                print(f"Error extracting rating: {e}") 
        time.sleep(2)
        all_reviews.extend(page_reviews)  # <--- use extend since we are working with a dictionary
        print(f"Stored {len(review_data)} reviews from Page {page}.")
        if page > 2:
            print(f"Reached page limit {page}")
            break
        if not click_next_page(driver):
            print("No more pages to scrape reviews from")
            break
        page += 1
    driver.quit()
    return all_reviews
df_reviews = scrape_yelp_reviews(clean_yelp_url)
print(df_reviews)

review[]
Found 0 reviews.
No reviews found on this page.
[]


In [9]:
import pandas as pd
import os
def create_csv(df_reviews, file_path="yelp_reviews.csv"):
    """This function converts df_reviews into a csv file"""
    df = pd.DataFrame(df_reviews)
    
    file_exists = os.path.isfile(file_path)

    df.to_csv(file_path, mode='a', index=False, header=not pd.io.common.file_exists(file_path))

    return file_path

create_csv(df_reviews, file_path="yelp_reviews.csv")

'yelp_reviews.csv'